In [10]:
# Import required functions and classes
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image, read_image_as_pil
from sahi.utils.file import Path, increment_path, list_files, save_json, save_pickle, download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict, agg_prediction, get_prediction_batched, get_sliced_prediction_batched 
from sahi.prediction import visualize_object_predictions
from IPython.display import Image
from numpy import asarray
import cv2
import os

In [18]:
# Download YOLOv8-S model to 'models/yolov8s.pt'
yolov11_model_path = 'models/yolo11/last.pt'
yolov8_model_path = 'models/yolov8/last.pt'
#download_yolov8s_model(destination_path=yolov8_model_path)

In [41]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='ultralytics',
    model_path=yolov8_model_path,
    confidence_threshold=0.3,
    device="cuda:0", # or 'cpu'
)

In [42]:
sliced_detection_model = AutoDetectionModel.from_pretrained(
    model_type='ultralytics',
    model_path=yolov8_model_path,
    confidence_threshold=0.3,
    device="cuda:0", # or 'cpu'
)

In [15]:
import cv2
from PIL import Image

def get_slice_parameters(object_density):
    
    #image_path = "test_data/0000006_06773_d_0000018.jpg"
    #image = Image.open(image_path).convert("RGB")
    #image_width, image_height  = image.size
    #slice_width = image_width
    #slice_height = image_height
    #overlap_width_ratio = 0.0
    #overlap_height_ratio = 0.0

    if object_density >= 50:
        slice_width = 512
        slice_height = 512
        overlap_width_ratio = 0.5
        overlap_height_ratio = 0.5
    elif 25 <= object_density < 50:
        slice_width = 512
        slice_height = 512
        overlap_width_ratio = 0.25
        overlap_height_ratio = 0.25
    elif 10 <= object_density < 25:
        slice_width = 512
        slice_height = 512
        overlap_width_ratio = 0.15
        overlap_height_ratio = 0.15
    else:
        return

    return slice_width, slice_height, overlap_width_ratio, overlap_height_ratio

In [16]:
# Function to get image details by image_id
def get_image_id(coco_data, image_name):
    for image in coco_data["images"]:
        file_name = Path(image['file_name']).stem
        if file_name == image_name:
            return image['id']
    return None

In [27]:
import os
import cv2
import json
import numpy as np
from tqdm import tqdm
import time
from sahi.prediction import ObjectPrediction, PredictionResult

# export visualization
def predict_sliced_images(input_folder, dataset_json_path, detection_model, sliced_detection_model):
    """
    Processes all image files in input_folder:
      - Runs predictions using get_prediction function and detection_model.
      - Saves annotated images with bounding boxes in output_folder.
      - Saves prediction details as JSON files in output_folder.
    
    Parameters:
      input_folder (str): Path to the folder containing images.
      output_folder (str): Path to the folder where results will be saved.
      detection_model: Your detection model used for prediction.
    """
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    if dataset_json_path:
        with open(dataset_json_path, "r") as file:
            data = json.load(file)
    
    #color = (0, 255, 0)  # original annotations in green
    visual_bbox_gt_thickness = 3
    visual_bbox_thickness = 2
    visual_text_size = 0.5
    visual_text_thickness = 1
    visual_hide_labels = False
    visual_hide_conf = False
    visual_export_format = 'png'
    sliced_predictions = []
    image_ids = []
    coco_json = []
    
    # Loop over files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_folder, filename)
            image_as_pil = read_image_as_pil(image_path)
            filename_without_ext = Path(filename).stem
            
            print("*****************************************")
            print("File Name", filename_without_ext)

            img_id = get_image_id(data, filename_without_ext)
            #image_ids.append(image_id)
            
            # Get predictions from your detection model

            
            time_start = time.time()
            prediction = get_prediction(image_path, detection_model) #changes
            time_end = time.time() - time_start
            #print(f"Intial Prediction Performed in {time_end} seconds")
            print("Intial Prediction time is: {:.2f} ms".format(time_end * 1000))
            
            object_density = len(prediction.object_prediction_list)
            print("Object Density:", object_density)
            
            if object_density > 10:
                slice_width, slice_height, overlap_width_ratio, overlap_height_ratio = get_slice_parameters(object_density)
    
                print("********* Slice Parameters ***********")
                print("Slice Width: ", slice_width)
                print("Slice Height: ", slice_height)
                print("Overlap Width Ratio: ", overlap_width_ratio)
                print("Overlap Height Ratio: ", overlap_height_ratio)

                time_start_slice = time.time()
                result_sahi = get_sliced_prediction(
                    image_path,
                    sliced_detection_model,
                    slice_height = slice_height,
                    slice_width = slice_width,
                    overlap_height_ratio = overlap_height_ratio,
                    overlap_width_ratio = overlap_width_ratio,
                    postprocess_type = "OptNMS",
                    verbose = 2
                )
                time_end_slice = time.time() - time_start_slice
                #print(f"Prediction Performed in {time_end1} seconds")
                print("Sliced Prediction time is: {:.2f} ms".format(time_end_slice * 1000))
                
                coco_prediction = result_sahi.to_coco_predictions(image_id=img_id)

                for idx, predict in enumerate(coco_prediction):
                    if coco_prediction[idx]["bbox"]:
                            coco_json.append(predict)
                    
                sliced_predictions.append(result_sahi)
                
                visualize_object_predictions(
                    np.ascontiguousarray(image_as_pil),
                    object_prediction_list=result_sahi.object_prediction_list,
                    rect_th=visual_bbox_thickness,
                    text_size=visual_text_size,
                    text_th=visual_text_thickness,
                    hide_labels=visual_hide_labels,
                    hide_conf=visual_hide_conf,
                    output_dir=save_dir,
                    file_name=filename_without_ext,
                    export_format=visual_export_format,
                )
                
            else:
                print("Prediction time is: {:.2f} ms".format(time_end * 1000))
                
                coco_prediction = prediction.to_coco_predictions(image_id=img_id)

                for idx, predict in enumerate(coco_prediction):
                    if coco_prediction[idx]["bbox"]:
                            coco_json.append(predict)
                    
                sliced_predictions.append(prediction)
                
                visualize_object_predictions(
                    np.ascontiguousarray(image_as_pil),
                    object_prediction_list=prediction.object_prediction_list,
                    rect_th=visual_bbox_thickness,
                    text_size=visual_text_size,
                    text_th=visual_text_thickness,
                    hide_labels=visual_hide_labels,
                    hide_conf=visual_hide_conf,
                    output_dir=save_dir,
                    file_name=filename_without_ext,
                    export_format=visual_export_format,
                )
        total_time = time_end + time_end_slice
                 
    if dataset_json_path:
        save_path = str(save_dir / "result.json")
        save_json(coco_json, save_path)
        print(f"Prediction results are successfully exported to {save_dir}")
    print(f"Prediction Completed Sucessfully: {len(sliced_predictions)} images")
    print("Total Prediction time is: {:.2f} ms".format(total_time * 1000))
    return sliced_predictions

In [32]:
# Example usage: Adaptive-Optimized-NMS
source_folder = './single_test/images'
json_path = "./subset_visdrone_test_990.json"
result_preds_adapt_opt_nms_iou = predict_sliced_images(source_folder, json_path, detection_model, sliced_detection_model)

*****************************************
File Name 0000006_06773_d_0000018
Intial Prediction time is: 27.44 ms
Object Density: 37
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
POST PROCESS:  OptNMS
Performing prediction on 8 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [848, 0]
CLASS TYPE:  False
Original Prediction Count 47
****non_class_agnostic****
Filtered Prediction:  19
Final Bounding Box Count: 12
Filtered Prediction:  2
Final Bounding Box Count: 2
Filtered Prediction:  19
Final Bounding Box Count: 14
Filtered Prediction:  5
Final Bounding Box Count: 4
Filtered Prediction:  2
Final Bounding Box Count: 2
Slicing performed in 0.005987405776977539 seconds.
Prediction performed in 0.07281327247619629 seconds.
Sliced Prediction time is: 77.96 ms
Prediction results are successfully exported to sliced_predictions/exp70
Prediction Completed Sucessfully: 1 images
Total Prediction time is: 105.40 ms


In [30]:
# Example usage: Adaptive-Optimized-NMS
source_folder = './single_test/images'
json_path = "./subset_visdrone_test_990.json"
result_preds_adapt_opt_nms_iou = predict_sliced_images(source_folder, json_path, detection_model, sliced_detection_model)

*****************************************
File Name 0000006_06773_d_0000018
Intial Prediction time is: 27.89 ms
Object Density: 37
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
POST PROCESS:  OptNMS
Performing prediction on 8 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [848, 0]
CLASS TYPE:  False
Original Prediction Count 47
****non_class_agnostic****
Filtered Prediction:  19
Final Bounding Box Count: 12
Filtered Prediction:  2
Final Bounding Box Count: 2
Filtered Prediction:  19
Final Bounding Box Count: 14
Filtered Prediction:  5
Final Bounding Box Count: 4
Filtered Prediction:  2
Final Bounding Box Count: 2
Slicing performed in 0.007994413375854492 seconds.
Prediction performed in 0.07484292984008789 seconds.
Sliced Prediction time is: 80.26 ms
Prediction results are successfully exported to sliced_predictions/exp69
Prediction Completed Sucessfully: 1 images
Total Prediction time is: 108.15 ms


In [23]:
# Example usage: Adaptive-Optimized-NMS
source_folder = './single_test/images'
json_path = "./subset_visdrone_test_990.json"
result_preds_adapt_opt_nms_iou = predict_sliced_images(source_folder, json_path, detection_model, sliced_detection_model)

*****************************************
File Name 0000006_06773_d_0000018
Intial Prediction time is: 24.98 ms
Object Density: 37
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
POST PROCESS:  OptNMS
Performing prediction on 8 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [848, 0]
CLASS TYPE:  False
Original Prediction Count 45
****non_class_agnostic****
Filtered Prediction:  26
Final Bounding Box Count: 17
Filtered Prediction:  2
Final Bounding Box Count: 2
Filtered Prediction:  13
Final Bounding Box Count: 8
Filtered Prediction:  3
Final Bounding Box Count: 2
Filtered Prediction:  1
Final Bounding Box Count: 1
Slicing performed in 0.006063938140869141 seconds.
Prediction performed in 0.06545186042785645 seconds.
Sliced Prediction time is: 71.31 ms
Prediction results are successfully exported to sliced_predictions/exp67
Prediction Completed Sucessfully: 1 images
Total Prediction time is: 96.30 ms


In [35]:
# Example usage: Adaptive-Optimized-NMS-IoU
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
result_preds_nms_iou_adaptive = predict_sliced_images(source_folder, json_path, detection_model, sliced_detection_model)

*****************************************
File Name 0000207_00300_d_0000004
Intial Prediction time is: 18.93 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
POST PROCESS:  OptNMS
Performing prediction on 6 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [0, 28]
CLASS TYPE:  False
Original Prediction Count 89
****non_class_agnostic****
Filtered Prediction:  8
Final Bounding Box Count: 5
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  51
Final Bounding Box Count: 28
Filtered Prediction:  5
Final Bounding Box Count: 3
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  15
Final Bounding Box Count: 6
Filtered Prediction:  5
Final Bounding Box Count: 3
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  2
Final Bounding Box Count: 1
Slicing performed in 0.0031630992889404297 seconds.
Prediction performed in 0.062881946563

In [39]:
# Example usage: Adaptive-Optimized-NMS-IoU
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
result_preds_nms_iou_adaptive_1 = predict_sliced_images(source_folder, json_path, detection_model, sliced_detection_model)

*****************************************
File Name 0000207_00300_d_0000004
Intial Prediction time is: 91.15 ms
Object Density: 23
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
POST PROCESS:  OptNMS
Performing prediction on 6 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [0, 28]
CLASS TYPE:  False
Original Prediction Count 101
****non_class_agnostic****
Filtered Prediction:  8
Final Bounding Box Count: 5
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  69
Final Bounding Box Count: 40
Filtered Prediction:  2
Final Bounding Box Count: 2
Filtered Prediction:  2
Final Bounding Box Count: 1
Filtered Prediction:  10
Final Bounding Box Count: 7
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  2
Final Bounding Box Count: 1
Filtered Prediction:  4
Final Bounding Box Count: 2
Slicing performed in 0.0030100345611572266 seconds.
Prediction performed in 0.11631083488

In [43]:
# Example usage: Adaptive-Optimized-NMS-IoU
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
result_preds_nms_iou_adaptive_2 = predict_sliced_images(source_folder, json_path, detection_model, sliced_detection_model)

*****************************************
File Name 0000207_00300_d_0000004
Intial Prediction time is: 75.60 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
POST PROCESS:  OptNMS
Performing prediction on 6 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [0, 28]
CLASS TYPE:  False
Original Prediction Count 101
****non_class_agnostic****
Filtered Prediction:  8
Final Bounding Box Count: 5
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  69
Final Bounding Box Count: 40
Filtered Prediction:  2
Final Bounding Box Count: 2
Filtered Prediction:  2
Final Bounding Box Count: 1
Filtered Prediction:  10
Final Bounding Box Count: 7
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  2
Final Bounding Box Count: 1
Filtered Prediction:  4
Final Bounding Box Count: 2
Slicing performed in 0.0031821727752685547 seconds.
Prediction performed in 0.11693334579

In [44]:
#USING Adaptive-Size METHOD (15 images)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp73/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.49s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.191
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.309
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.221
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.256
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.470
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.385
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.330
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [40]:
#USING Adaptive-Size METHOD (15 images)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp72/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.46s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.191
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.309
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.221
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.255
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.472
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.402
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.142
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.330
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [36]:
#USING Adaptive-Size METHOD (15 images)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp71/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.50s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.178
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.297
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.238
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.441
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.391
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.129
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.298
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [24]:
#USING Adaptive-Size METHOD
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './sliced_predictions/exp67/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.246
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.521
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.216
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.832
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.544
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.287
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.267
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.281
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [29]:
#USING Adaptive-Size METHOD
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './sliced_predictions/exp68/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.305
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.651
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.256
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.832
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.667
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.545
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.250
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.372
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [31]:
#USING Adaptive-Size METHOD
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './sliced_predictions/exp69/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.305
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.651
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.256
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.832
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.667
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.545
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.250
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.372
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [45]:
#USING Adaptive-Size METHOD
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './sliced_predictions/exp70/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.305
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.651
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.256
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.832
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.667
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.545
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.250
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.372
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [ ]:
# Example usage: Adaptive-NMS
source_folder = 'test_data'
json_path = "../data/VisDrone2COCO/COCO/annotations/visdrone_coco_test.json"
result_preds_nms = predict_sliced_images(source_folder, json_path, detection_model)